# Processamento de Dados de Vendas (2022-2024)
Este notebook é dedicado ao tratamento abrangente de dados de vendas abrangendo o período de 2022 a 2024. Sua função principal é padronizar e corrigir os dados brutos de acordo com as regras comerciais estabelecidas, garantindo a integridade e a confiabilidade do conjunto de dados.

In [231]:
# Defininindo o arquivo de trabalho
arquivo = '../data/raw/vendas_22-24.csv'

In [232]:
# Importação das bibliotecas utilizadas.
import pandas as pd
import statistics as sts

In [233]:
# Carregamento do arquivo de dados a ser utilizado.
dados = pd.read_csv(arquivo, delimiter=';')
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Data             10000 non-null  object 
 1   Loja             9672 non-null   object 
 2   Produto          10000 non-null  object 
 3   Categoria        10000 non-null  object 
 4   Quantidade       9691 non-null   float64
 5   Preço Unitário   9688 non-null   float64
 6   Total Venda      7979 non-null   float64
 7   Cliente_ID       9732 non-null   float64
 8   Forma_Pagamento  9684 non-null   object 
dtypes: float64(4), object(5)
memory usage: 703.3+ KB


In [234]:
# Verificando o padrão de colunas
dados.head()

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento
0,2022-01-18,Loja B,Refrigerante,Bebidas,3.0,7.50,22.50,6727.0,Dinheiro
1,2024-12-29,Loja B,Leite,Alimentos,5.0,14.74,73.70,2120.0,Transferência
2,2023-03-21,Loja C,Refrigerante,Bebidas,2.0,NaN,72.60,5353.0,Transferência
3,2022-08-03,Loja C,Feijão,Alimentos,4.0,26.75,107.00,6835.0,Transferência
4,2022-06-29,Loja C,Feijão,Alimentos,7.0,30.47,213.29,5824.0,Dinheiro


In [235]:
# contagem de valores nulos.
dados.isnull().sum()

Data                  0
Loja                328
Produto               0
Categoria             0
Quantidade          309
Preço Unitário      312
Total Venda        2021
Cliente_ID          268
Forma_Pagamento     316
dtype: int64

### Regra de negócio Loja

In [236]:
# Verificação dos dados de loja
dados.groupby('Loja').size()

Loja
Loja A    3238
Loja B    3215
Loja C    3219
dtype: int64

In [237]:
# Preenche o valor da loja com o valor de maior recorrência
dados.fillna({'Loja': 'Loja A'}, inplace=True)

### Regra de negócio Quantidade

In [238]:
# Nova contagem de nulos
dados.isnull().sum()

Data                  0
Loja                  0
Produto               0
Categoria             0
Quantidade          309
Preço Unitário      312
Total Venda        2021
Cliente_ID          268
Forma_Pagamento     316
dtype: int64

In [239]:
# Calculo da mediana para preenchimento de valores incorretos ou nulos
medianaQtd = sts.median(dados['Quantidade'])
medianaQtd

2.0

In [240]:
# Preenchimento de valores nulos
dados['Quantidade'].fillna(medianaQtd, inplace=True)

C:\Users\jamsilva\AppData\Local\Temp\ipykernel_17832\3798690500.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dados['Quantidade'].fillna(medianaQtd, inplace=True)


In [241]:
# Filtro de valores fora da regra de negócio
dados.loc[(dados['Quantidade'] < 1) | (dados['Quantidade'] > 100)]

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento
11,2024-10-26,Loja A,Refrigerante,Bebidas,-4.0,16.74,NaN,5087.0,Cartão
67,2024-05-27,Loja C,Detergente,Limpeza,-5.0,13.36,NaN,8280.0,Dinheiro
77,2024-07-26,Loja B,Detergente,Limpeza,-4.0,32.10,NaN,8789.0,Pix
166,2024-02-29,Loja B,Refrigerante,Bebidas,-1.0,42.32,NaN,1634.0,Pix
179,2024-06-13,Loja A,Detergente,Limpeza,-5.0,42.95,NaN,2761.0,Cartão
...,...,...,...,...,...,...,...,...,...
9867,2024-01-08,Loja C,Açúcar,Alimentos,-4.0,44.72,NaN,9766.0,Cartão
9873,2023-12-31,Loja C,Pasta de Dente,Higiene,-1.0,28.69,NaN,8971.0,Dinheiro
9888,2024-06-15,Loja B,Shampoo,Higiene,-2.0,24.56,NaN,3315.0,Cartão
9985,2022-08-10,Loja A,Refrigerante,Bebidas,-5.0,7.88,NaN,7589.0,Cartão


In [242]:
# Correção de valores incorretos com base na regra de negócio
dados.loc[(dados['Quantidade'] < 1) | (dados['Quantidade'] > 100)] = medianaQtd

In [243]:
# Nova verificação de valores fora da regra
dados.loc[(dados['Quantidade'] < 1) | (dados['Quantidade'] > 100)]

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento


In [244]:
# Contagem de nulos
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Data             10000 non-null  object 
 1   Loja             10000 non-null  object 
 2   Produto          10000 non-null  object 
 3   Categoria        10000 non-null  object 
 4   Quantidade       10000 non-null  float64
 5   Preço Unitário   9697 non-null   float64
 6   Total Venda      8346 non-null   float64
 7   Cliente_ID       9740 non-null   float64
 8   Forma_Pagamento  9696 non-null   object 
dtypes: float64(4), object(5)
memory usage: 703.3+ KB


### Regra de negócio Preço

In [251]:
mediaPreco = dados['Preço Unitário'].mean()
mediaPreco

np.float64(22.41667216664948)

In [250]:
dados.loc[(dados['Preço Unitário'] < 0.50) | (dados['Preço Unitário'] > 500.0)]

,Data,Loja,Produto,Categoria,Quantidade,Preço Unitário,Total Venda,Cliente_ID,Forma_Pagamento
27,2023-01-20,Loja A,Cerveja,Bebidas,9.0,-10.14,NaN,2471.0,Transferência
31,2023-12-09,Loja C,Pasta de Dente,Higiene,9.0,-41.96,NaN,8619.0,Cartão
57,2024-05-26,Loja A,Refrigerante,Bebidas,5.0,-3.20,NaN,1567.0,Dinheiro
91,2023-07-13,Loja B,Leite,Alimentos,2.0,-18.86,NaN,6228.0,Dinheiro
96,2024-01-17,Loja A,Feijão,Alimentos,3.0,-12.44,NaN,4016.0,Pix
...,...,...,...,...,...,...,...,...,...
9830,2024-06-13,Loja A,Detergente,Limpeza,4.0,-37.82,NaN,2850.0,Pix
9906,2024-07-19,Loja B,Arroz,Alimentos,6.0,-33.67,NaN,3402.0,Pix
9956,2023-03-27,Loja A,Cerveja,Bebidas,1.0,-15.65,NaN,1950.0,Cartão
9966,2022-07-18,Loja C,Sabão em Pó,Limpeza,6.0,-20.57,NaN,9116.0,Transferência


In [252]:
dados['Preço Unitário'].fillna(mediaPreco, inplace=True)
dados.loc[(dados['Preço Unitário'] < 0.50) | (dados['Preço Unitário'] > 500.0)] = mediaPreco

C:\Users\jamsilva\AppData\Local\Temp\ipykernel_17832\1929562576.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dados['Preço Unitário'].fillna(mediaPreco, inplace=True)


In [254]:
dados['Preço Unitário'].describe()

count    10000.000000
mean        24.661652
std         13.959734
min          1.500000
25%         13.157500
50%         22.595000
75%         36.330000
max         50.000000
Name: Preço Unitário, dtype: float64